# Final Project for Applied Physics in Programming
By: Eveliina Hampus, DIN22SP

1. Import necessary libraries. 

Remember to install libraries with %pip install or properly via CLI with pip install.
Notice that Oceanborn is depentant from Matplotlib.

In [3]:
import numpy as np # For calculations
import matplotlib.pyplot as plt # For basic plotting
import pandas as pd # For data manipulation
import sympy as sm # For symbolic calculations
import scipy as sc # For scientific processing
import matplotlib as mpl # To support oceanborn libary
import seaborn as sns # For more informative plots, requires matplotlib to be imported first
from scipy.signal import butter,filtfilt # For filtering
from math import radians, cos, sin, asin, sqrt #
from sympy.abc import x #